# **TP3:  Numerical Differentiations and Integrations**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import sympy
import scipy
from collections.abc import Callable

## **Exo1:**

Use the forward-difference formulas and backward-difference formulas to determine
each missing entry in the following tables.

### **(a)**

\begin{array}{l|l|l}
x & f(x) & f^{\prime}(x) \\
\hline 0.5 & 0.4794 & 0.852 \\
\hline 0.6 & 0.5646 & 0.852 \\
\hline 0.7 & 0.6442 & 0.796 \\
\hline
\end{array}
We used the forward difference formula to compute $f^{\prime}(0.5)$ and the backward difference formula to compute $f^{\prime}(0.6)$ and $f^{\prime}(0.7)$. If the forward difference formula were used to compute $f^{\prime}(0)$, the result would be 0.796 .

### **(b)**

\begin{array}{l|l|l}
x & f(x) & f^{\prime}(x) \\
\hline 0.0 & 0.00000 & 3.707 \\
\hline 0.2 & 0.74140 & 3.152 \\
\hline 0.4 & 1.3718 & 3.152 \\
\hline
\end{array}
Forward difference was used for the first two, and backward difference for the last one. If backward difference were used for the second one, the answer would be 3.707.

## **Exo2:**

Use the most accurate three-point formula to determine each missing entry in the
following tables.

### **(a)**

\begin{array}{l|l|l}
x & f(x) & f^{\prime}(x) \\
\hline 1.1 & 9.025013 & 17.769705 \\
\hline 1.2 & 11.02318 & 22.193635 \\
\hline 1.3 & 13.46374 & 27.107350 \\
\hline 1.4 & 16.44465 & 32.150850 \\
\hline
\end{array}

For the endpoints of the tables, we use 3-Point Endpoint Formula. The other approximations come from 3-Point Midpoint formula.

### **(b)**

\begin{array}{l|l|l}
x & f(x) & f^{\prime}(x) \\
\hline 8.1 & 16.94410 & 3.092050 \\
\hline 8.3 & 17.56492 & 3.116150 \\
\hline 8.5 & 18.19056 & 3.139975 \\
\hline 8.7 & 18.82091 & 3.163525 \\
\hline
\end{array}

## **Exo3:**

In [24]:
A_0_h = 2.356194
A_0_h_2 = -0.4879837
A_0_h_4 = -0.8815732
A_0_h_8 = -0.9709157

A_1_h = A_0_h_2 + (A_0_h_2 - A_0_h)/(2**2 - 1)
A_1_h_2 = A_0_h_4 + (A_0_h_4 - A_0_h_2)/(2**2 - 1)
A_1_h_4 = A_0_h_8 + (A_0_h_8 - A_0_h_4)/(2**2 - 1)
print(A_1_h, A_1_h_2, A_1_h_4)

A_2_h = A_1_h_2 + (A_1_h_2 - A_1_h)/(2**4 - 1)
A_2_h_2 = A_1_h_4 + (A_1_h_4 - A_1_h_2)/(2**4 - 1)
print(A_2_h, A_2_h_2)

A_3_h = A_2_h_2 + (A_2_h_2 - A_2_h)/(2**6 - 1)
print(A_3_h)

-1.4360429333333333 -1.0127697 -1.0006965333333335
-0.9845514844444444 -0.9998916555555558
-1.0001351503350973


In [22]:
def Richardson_Extrapolation(f: Callable[[float], float],
                             x: float,
                             h: float,
                             n: int = 9,
                             rtol: float = 1e-10
                            ) -> tuple[float, pd.DataFrame]:
    N = n + 1
    D = np.full(shape=(N,N), fill_value=np.nan, dtype=np.float64)
    D[0, 0] = 0.5 * (f(x + h) - f(x - h)) / h
    for i in range(1, N, 1):
        h = 0.5 * h
        D[i, 0] = 0.5 * (f(x + h) - f(x - h)) / h
        I = i + 1
        p = 1
        for j in range(1, I, 1):
            p = 4*p
            D[i, j] = D[i, j-1] + (D[i, j-1] - D[i-1, j-1]) / (p-1)
        if abs(D[i,i] - D[i-1, i-1]) < rtol:
            break
    d = D[i,i]
    columns = [f'O(h^{2*(k+1)})' for k in range(0, I, 1)]
    D = pd.DataFrame(data=D[:I, :I], columns=columns)
    return (d, D)

In [23]:
def f(x): return np.log(1+x)
d, D = Richardson_Extrapolation(f=f, x=1, h=0.1)
print(f"d = {d:.10f}")
D.fillna(value='', inplace=True)
print(D)

d = 0.5000000000
     O(h^2) O(h^4) O(h^6) O(h^8)
0  0.500417                     
1  0.500104    0.5              
2  0.500026    0.5    0.5       
3  0.500007    0.5    0.5    0.5


## **Exo4**

## **Exo5**

Approximate the following integrals using the Trapezoidal rule, Simpson’s rule and Midpoint rule.

### **(a)**

With $f(x)=x^2 \ln x$ and $h=(1.5-1)=0.5$ the Trapezoid Rule gives
$$
\begin{aligned}
\int_1^{1.5} x^2 \ln x d x & \approx \frac{h}{2}[f(1)+f(1.5)] \\
& =\frac{0.5}{2}\left[1^2 \ln 1+1.5^2 \ln 1.5\right] \approx 0.228
\end{aligned}
$$
The absolute error is
$$
|E|=\frac{h^3}{12}\left|f^{\prime \prime}(\xi)\right|
$$
for some $\xi \in[1,1.5]$. Note that
$$
\left|f^{\prime \prime}(x)\right|=3+2 \ln x
$$
is increasing $\forall x>1$ so
$$
\left|f^{\prime \prime}(\xi)\right| \leq\left|f^{\prime \prime}(1.5)\right|=3+2 \ln 1.5 .
$$
Thus
$$
|E|=\frac{0.5^3}{12}\left|f^{\prime \prime}(\xi)\right| \leq \frac{0.5^3}{12}(3+2 \ln 1.5) \approx 0.040 .
$$
We can conclude:
$$
\int_1^{1.5} x^2 \ln x d x=0.228 \pm 0.040
$$

In [34]:
def TrapezoidalRule(f, a, b):
    h = b - a
    return (h * 0.5 * (f(a) + f(b)))

In [37]:
f_a = lambda x : x**2 * np.log(x)
TrapezoidalRule(f = f_a, a = 1, b = 1.5)

0.22807412331084248

In [36]:
def SimpsonRule(f, a, b):
    h = (b - a)/2
    c = a + h
    return (h/3 * (f(a) + 4*f(c) + f(b))) 

In [38]:
SimpsonRule(f = f_a, a = 1, b = 1.5)

0.19224530741309842

In [39]:
def MidpointRule(f, a, b):
    h = (b - a)/2
    x_0 = a + h
    return 2*h*f(x_0)

In [40]:
MidpointRule(f = f_a, a = 1, b = 1.5)

0.1743308994642264

### **(b)**

In [42]:
f_b = lambda x : np.exp(1)**(3*x) * np.sin(2*x)
print(TrapezoidalRule(f = f_b, a = 0, b = np.pi/4))
print(SimpsonRule(f = f_b, a = 0, b = np.pi/4))
print(MidpointRule(f = f_b, a = 0, b = np.pi/4))

4.143259655194082
2.5836964032474845
1.8039147772741864


## **Exo6**

## **Exo9**

In [3]:
def CompositeTrapezoidal(f: Callable[[float], float], a:float, b:float, n:int) -> float:
    """
    --------------------
    Approximate the integral of 'f(x)' from 'a' to 'b' using Composite Trapezoidal rule/formula
    
    Parameters
    --------------------
    'f' : 'callable'
        function that we want to integrate
    'a' : 'float'
        lower limit of the integral
    'b' : 'float'
        upper limit of the integral
    'n' : 'int'
        (n+1) be the number of nodes
        
    Return
    -------------------
    'A' : 'float'
        the approximated value of the integral
    """
    h = (b-a) / n
    f_0 = f(a) + f(b)
    f_i = 0
    for i in range(1, n):
        x = a + i*h
        f_i = f_i + f(x)
    A = 0.5 * h * (f_0 + 2*f_i)
    return A

## **Exo10**

Use the Composite Trapezoidal rule with the indicated values of n to approximate the
following integrals.

### **(a)**

In [69]:
f_10_a = lambda x: x * np.log(x)
CompositeTrapezoidal(f = f_10_a, a = 1, b = 2, n = 4)

0.639900477687986

### **(b)**

In [70]:
f_10_b = lambda x: x / (x**2 + 4)
CompositeTrapezoidal(f = f_10_b, a = 1, b = 3, n = 8)

0.4769768665147184

## **Exo11**

In [4]:
def CompositeSimpson(f: Callable[[float], float], a:float, b:float, n:int) -> float:
    """
    --------------------
    Approximate the integral of 'f(x)' from 'a' to 'b' using Composite Simpson rule/formula
    
    Parameters
    --------------------
    'f' : 'callable'
        function that we want to integrate
    'a' : 'float'
        lower limit of the integral
    'b' : 'float'
        upper limit of the integral
    'n' : 'int'
        (n+1) be the number of nodes
        
    Return
    -------------------
    'A' : 'float'
        the approximated value of the integral
    """
    h = (b-a) / n
    f_0 = f(a) + f(b)
    f_1 = 0
    f_2 = 0
    for i in range(1, n):
        x = a + i*h
        if (i % 2 == 0):
            f_2 = f_2 + f(x)
        else:
            f_1 = f_1 + f(x)
    A = h * (f_0 + 2*f_2 + 4*f_1) / 3
    return A

## **Exo12**

Use the Composite Simpson’s rule with the indicated values of n to approximate the
following integrals.

### **(a)**

In [72]:
f_12_a = lambda x: x**3 * np.exp(x)
CompositeSimpson(f = f_12_a, a = -2, b = 2, n = 4)

22.477125358234236

### **(b)**

In [67]:
f_12_b = lambda x: x**2 * np.cos(x)
CompositeSimpson(f = f_12_b, a = 1, b = 3, n = 6)
print(CompositeSimpson(f = f_12_b, a = 0, b = np.pi, n = 6))

-6.274868388453119


## **Exo13**

In [5]:
def CompositeMidpoint(f: Callable[[float], float], a:float, b:float, n:int) -> float:
    """
    --------------------
    Approximate the integral of 'f(x)' from 'a' to 'b' using Composite Midpoint rule/formula
    
    Parameters
    --------------------
    'f' : 'callable'
        function that we want to integrate
    'a' : 'float'
        lower limit of the integral
    'b' : 'float'
        upper limit of the integral
    'n' : 'int'
        (n+1) be the number of nodes
        
    Return
    -------------------
    'A' : 'float'
        the approximated value of the integral
    """
    h = (b-a) / (n+2)
    f_2 = 0
    for i in range(0, n+3):
        x = a + i*h
        if (i % 2 != 0):
            f_2 = f_2 + f(x)
    A = 2 * h * f_2
    return A

## **Exo14**

Use the Composite Midpoint rule with $n + 2$ subintervals to approximate the following
integrals.

### **(a)**

In [93]:
f_14_a = lambda x: 2 / (x**2 + 4)
CompositeMidpoint(f = f_14_a, a = 0, b = 2, n = 6)

0.7867001295984857

### **(b)**

In [94]:
f_14_b = lambda x: np.exp(2*x) * np.sin(3*x)
CompositeMidpoint(f = f_14_b, a = 0, b = 2, n = 8)

-14.998476941632312

## **Exo15**

In [97]:
f_15 = lambda x: x**2 * np.exp(-x**2)
print(CompositeTrapezoidal(f = f_15, a = 0, b = 2, n = 8))
print(CompositeSimpson(f = f_15, a = 0, b = 2, n = 8))
print(CompositeMidpoint(f = f_15, a = 0, b = 2, n = 8))

0.42158203719810194
0.4227161879339765
0.42417920849962915


## **Exo16**

### **(a)**

In [8]:
h_CT = np.sqrt(6*10**(-5)/0.03125)
n_CT = 2/h_CT
print(h_CT, n_CT)

0.04381780460041329 45.64354645876384


In [9]:
f_16 = lambda x: 1 / (x + 4)
CompositeTrapezoidal(f = f_16, a = 0, b = 2, n = 46)

0.40547057780408446

### **(b)**

In [10]:
h_CS = (90*10**(-5)/0.0234375)**(1/4)
n_CS = 2/h_CS
print(h_CS, n_CS)

0.44267276788012866 4.518010018049224


In [12]:
CompositeSimpson(f = f_16, a = 0, b = 2, n = 6)

0.4054663745840217

### **(c)**

In [14]:
h_CM = np.sqrt(3*10**(-5)/0.03125)
n_CM = (2/h_CM) - 2
print(h_CM, n_CM)

0.03098386676965934 62.549722436790276


In [17]:
CompositeMidpoint(f = f_16, a = 0, b = 2, n = 64)

0.40545979433290036

## **Exo17**